# Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd

# Import the dataset

In [2]:
Dataset = pd.read_csv('OneDrive/Desktop/Week 3 - Data Cleaning (Pandas) - 2/chipotle.tsv',sep='\t')

# Knowing our dataset

In [3]:
Dataset.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


# Checking datatypes of each column

In [4]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


# Handling missing values
As we can see there are around 1400 missing values in choice_description column. choice_description column is a categorical column so we can not replace those null values with mean or median of the column. After studying the dataset, we can see that the there are null values in choice_description column for only those items in item_name for which no choices are available. Therefore, we can replace the null values with 'No Choices' String value.

In [5]:
Dataset.isnull().sum()

order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

In [6]:
Dataset['choice_description'] = Dataset['choice_description'].fillna('No choice')

# Handling inconsistency in data
Now, If we check the dtype of item_price, we can see that it is object because there is dollar sign attached to the price value. If we want to perform some statistical analysis, we have to change the dtype of item_price column to 'float64'.Also we have to remove the dollar sign. We can use the replace method of python to do that. To change the dtype we use 'astype' method.

In [7]:
Dataset['item_price'] = Dataset['item_price'].str.replace('$', '').astype(float)

# Analyzing choice_desciption column
If we look at the choice_description column, we can clearly see that there are multiple choices for a single item which should be handled properly for accurate analysis. What we can do in this case is create a column for every unique item in choice_description column and mark their presence by 0 or 1 according to their corresponding value in choice_description column just like one-hot encoding.

In [8]:
def extract_items(choice_description):
    if pd.isnull(choice_description):
        return []
    else:
        # Assuming items are enclosed in square brackets
        return [item.strip(" []") for item in choice_description.split(",")]
Dataset['parsed_items'] = Dataset['choice_description'].apply(extract_items)
unique_items = set(item for sublist in Dataset['parsed_items'] for item in sublist)
for item in unique_items:
    Dataset[item] = Dataset['parsed_items'].apply(lambda x: 1 if item in x else 0)
Dataset.drop(['choice_description', 'parsed_items'], axis=1, inplace=True)

# Data integrity check for order_id column
To cross-reference the Order ID column for integrity, we can check for irregularities or patterns by examining the unique values in the 'order_id' column. 

In [9]:
# Check unique values in the 'order_id' column
unique_order_ids = Dataset['order_id'].unique()

# Check for irregularities or patterns
# We can check if the order IDs follow a specific pattern or have consistent lengths
for order_id in unique_order_ids:
    print(f"Order ID: {order_id}, Length: {len(str(order_id))}")

# Additionally, we can check if there are any missing values in the 'order_id' column
missing_order_ids = Dataset['order_id'].isnull().sum()
print(f"Number of missing Order IDs: {missing_order_ids}")

Order ID: 1, Length: 1
Order ID: 2, Length: 1
Order ID: 3, Length: 1
Order ID: 4, Length: 1
Order ID: 5, Length: 1
Order ID: 6, Length: 1
Order ID: 7, Length: 1
Order ID: 8, Length: 1
Order ID: 9, Length: 1
Order ID: 10, Length: 2
Order ID: 11, Length: 2
Order ID: 12, Length: 2
Order ID: 13, Length: 2
Order ID: 14, Length: 2
Order ID: 15, Length: 2
Order ID: 16, Length: 2
Order ID: 17, Length: 2
Order ID: 18, Length: 2
Order ID: 19, Length: 2
Order ID: 20, Length: 2
Order ID: 21, Length: 2
Order ID: 22, Length: 2
Order ID: 23, Length: 2
Order ID: 24, Length: 2
Order ID: 25, Length: 2
Order ID: 26, Length: 2
Order ID: 27, Length: 2
Order ID: 28, Length: 2
Order ID: 29, Length: 2
Order ID: 30, Length: 2
Order ID: 31, Length: 2
Order ID: 32, Length: 2
Order ID: 33, Length: 2
Order ID: 34, Length: 2
Order ID: 35, Length: 2
Order ID: 36, Length: 2
Order ID: 37, Length: 2
Order ID: 38, Length: 2
Order ID: 39, Length: 2
Order ID: 40, Length: 2
Order ID: 41, Length: 2
Order ID: 42, Length: 2
O

# Item name standardization
To standardize the Item Name column and unify variations for better analysis, we can apply text processing techniques to clean and standardize the item names.
The code performs the following operations:
1.Converts all item names to lowercase for consistency.
2.Removes leading and trailing whitespaces.
3.Replaces hyphens with spaces.

In [10]:
Dataset['item_name'] = Dataset['item_name'].str.lower() 
Dataset['item_name'] = Dataset['item_name'].str.strip() 
Dataset['item_name'] = Dataset['item_name'].str.replace('-', ' ')

# Relationship between Quantity and Item_price
To perform a data integrity check to ensure that quantities and prices align with the corresponding items and descriptions, we can examine the dataset to identify any discrepancies.This code checks for records where the product of quantity and item price does not match the total price, highlighting potential inconsistencies.This analysis helps to identify records where the calculated total price does not align with the given quantity and item price.

In [11]:
Dataset['total_price'] = Dataset['quantity'] * Dataset['item_price']

# Check for inconsistencies between quantities and prices
inconsistent_records = Dataset[Dataset['total_price'] != Dataset['total_price']]

# Display inconsistent records
print("Inconsistent Records:")
print(inconsistent_records)

Inconsistent Records:
Empty DataFrame
Columns: [order_id, quantity, item_name, item_price, Diet Coke, Pineapple Orange Banana, Tomatillo-Green Chili Salsa (Medium), Fresh Tomato (Mild), Roasted Chili Corn (Medium), Guacamole, Roasted Chili Corn Salsa (Medium), Coca Cola, White Rice, Tomatillo Red Chili Salsa, Brown Rice, Sprite, Clementine, Roasted Chili Corn Salsa, Cheese, Lemonade, Tomatillo-Red Chili Salsa (Hot), Mountain Dew, Adobo-Marinated and Grilled Chicken, Tomatillo Red Chili (Hot), Peach Orange, Coke, Nestea, Cilantro-Lime Rice, Tomatillo Green Chili (Medium), Fajita Veggies, Vegetarian Black Beans, Grapefruit, Braised Carnitas, Rice, Adobo-Marinated and Grilled Steak, Fresh Tomato Salsa, Pomegranate Cherry, Fajita Vegetables, Black Beans, Fresh Tomato Salsa (Mild), Dr. Pepper, Salsa, Diet Dr. Pepper, No choice, Tomatillo Green Chili Salsa, Apple, Sour Cream, Braised Barbacoa, Lettuce, Pinto Beans, Blackberry, total_price]
Index: []

[0 rows x 52 columns]
